# pandas 데이터 파악과 조작

**분석할 데이터를 수집(확보)하면 데이터의 특징을 파악하고 다루기 쉽게 변형하는 작업을 수행해야 한다**

# #2. 데이터 조작(가공)

- 데이터 개수 세기 : count(), value_counts()
- 데이터 정렬 : sort_values(), sort_index()
- 데이터 집계 : 합계(sum()), 평균(mean()), 최대(max()), 최소(min())
- 데이터 삭제 : drop(axis=0/1)
- 결측치 처리 : dropna(axis=0/1, subset, inplace)
- 데이터 변경 : 
    - 자료형 변경 : astype()
    - 수치형 데이터를 범주형 데이터로 변경 : 
        - 구간을 지정하여 범주화 : cut(data, bins, labels)
        - 동일한 개수를 갖도록 범주화 : qcut(data, bins_num, labels)
- 행/열에 동일한 함수 적용 : apply()

---

In [1]:
import numpy as np
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

# 7. 행/열에 동일한 연산 적용 apply()

#### apply() 함수
- DataFrame의 행이나 열에 복잡한 연산을 vectorizing 할 수 있게 해주는 함수
- 매우 많이 활용되는 함수


**형식 : apply(반복적용할 함수, axis=0/1)**
- 0 : 열마다 반복
- 1 : 행마다 반복 
- 생략시 기본값 : 0

### 예제

In [5]:
# 예제 df 생성
df = pd.DataFrame({'a':[1,3,4,2,4], 'b':[2,3,1,4,5], 'c':[1,5,2,4,4]})
df

,a,b,c
0,1,2,1
1,3,3,5
2,4,1,2
3,2,4,4
4,4,5,4


### 데이터프레임의 각 열에 sum() 함수 적용

apply(함수, axis=0)

In [6]:
df.apply(np.sum, axis=0)

a    14
b    15
c    16
dtype: int64

In [8]:
df.sum()

a    14
b    15
c    16
dtype: int64

sum함수는 열 또는 행단위로 적용되는 함수여서 각 열별로 적용 됨

### 데이터프레임의 각 행에 sum() 함수 적용

In [11]:
df.apply(np.sum, axis=1)

0     4
1    11
2     7
3    10
4    13
dtype: int64

In [10]:
df.sum(axis=1)

0     4
1    11
2     7
3    10
4    13
dtype: int64

### 데이터프레임의 각 원소의 제곱값을 계산

- 컬럼별 모든 원소에 대하여 np.square 함수 적용

In [12]:
np.square([1,2,3])

array([1, 4, 9])

In [14]:
df

,a,b,c
0,1,2,1
1,3,3,5
2,4,1,2
3,2,4,4
4,4,5,4


In [13]:
df.apply(np.square) #모든 값을 제곱함

,a,b,c
0,1,4,1
1,9,9,25
2,16,1,4
3,4,16,16
4,16,25,16


- 행별 모든 원소에 대하여 np.square 함수 적용

In [15]:
df.apply(np.square, axis=1)

,a,b,c
0,1,4,1
1,9,9,25
2,16,1,4
3,4,16,16
4,16,25,16


## 사용자가 정의한 연산을 행/열단위 적용

### lambda() & apply()

- 데이터프레임의 기본 집계함수(sum, min, max, mean 등)들은 행/열 단위 벡터화 연산을 수행함
    - apply() 함수를 사용할 필요가 없음

- apply() 함수 사용은 복잡한 연산을 해결하기 위한 lambda 함수나 사용자 정의 함수를 각 열 또는 행에 일괄 적용시키기 위해 사용
    - lambda 함수로 필요한 연산 기능을 구현하고, apply()를 통해 열/행 단위로 적용

### 1회성 함수 lambda 함수를  apply()에 사용하는 예제

#### 예1. 집합데이터의 최대값과 최소값의 차이를 구하는 lambda 함수 정의

In [19]:
range = lambda x:x.max()-x.min()

- 정의한 lambda함수 dev() 적용

In [20]:
s = pd.Series([3,1,9,10,4])
range(s)

9

In [21]:
df

,a,b,c
0,1,2,1
1,3,3,5
2,4,1,2
3,2,4,4
4,4,5,4


In [22]:
df.apply(range) #컬럼별 최대최소의 차이

a    3
b    4
c    4
dtype: int64

In [23]:
df.apply(range, axis=1)

0    1
1    2
2    3
3    2
4    1
dtype: int64

In [24]:
df.apply(lambda x:x.max()-x.min(), axis=0)

a    3
b    4
c    4
dtype: int64

- 직접 연산을 통해 각 행의 최대값과 최소값 차이 계산

In [26]:
df.max() - df.min()

a    3
b    4
c    4
dtype: int64

In [27]:
df.max(axis=1) - df.min(axis=1)

0    1
1    2
2    3
3    2
4    1
dtype: int64

#### 예2. 데이터프레임 각 열의 데이터에 대한 범주별 빈도 계산

In [28]:
df

,a,b,c
0,1,2,1
1,3,3,5
2,4,1,2
3,2,4,4
4,4,5,4


In [29]:
df.a.value_counts()

a
4    2
1    1
3    1
2    1
Name: count, dtype: int64

In [31]:
for col in df.columns:
    print(df[col].value_counts())

a
4    2
1    1
3    1
2    1
Name: count, dtype: int64
b
2    1
3    1
1    1
4    1
5    1
Name: count, dtype: int64
c
4    2
1    1
5    1
2    1
Name: count, dtype: int64


In [32]:
df.apply(pd.value_counts)

,a,b,c
1,1.0,1,1.0
2,1.0,1,1.0
3,1.0,1,NaN
4,2.0,1,2.0
5,NaN,1,1.0


#### 예3. 각 열의 데이터에 대한 범주별 빈도 계산 후, NaN값은 0으로 변환하고 전체 데이터 타입을 정수로 변환

In [34]:
df.apply(pd.value_counts).fillna(0).astype(int)

,a,b,c
1,1,1,1
2,1,1,1
3,1,1,0
4,2,1,2
5,0,1,1


-----------------------------------------